In [7]:
from langchain.llms import OpenAI

llm = OpenAI(model_name = 'text-davinci-003', temperature=0)

text = "What would be a good company name for a company that makes colorful socks?"

print(llm(text))



Rainbow Socks Co.


Tracking token usage

In [8]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(model_name="text-davinci-003", n=2, best_of=2)

with get_openai_callback() as cb:
    result = llm('Tell me a joke')
    print(cb)

Tokens Used: 44
	Prompt Tokens: 4
	Completion Tokens: 40
Successful Requests: 1
Total Cost (USD): $0.00088


### Few shot learning
Learn and generalize from limited examples

In [9]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(input_variables=['query', 'answer'], template=example_template)

prefix = """
The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some examples:
"""

suffix = """
User: {query}
AI: 
"""

few_shot_prompt_tempalte = FewShotPromptTemplate(examples=examples,
                                                 example_prompt=example_prompt,
                                                 prefix=prefix,
                                                 suffix=suffix,
                                                 input_variables=['query'],
                                                 example_separator="\n\n")

from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

chat = ChatOpenAI(model_name='gpt-4', temperature=0.0)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_tempalte)
chain.run("What's the meaning of life?")

'42, according to "The Hitchhiker\'s Guide to the Galaxy". But if you\'re looking for a more philosophical answer, I\'m afraid you\'ll have to ask a philosopher. I\'m just an AI!'

### Examples with easy prompts:
* Text summarization
* Text translation
* Question answering

In [11]:
from langchain import PromptTemplate

template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=['question'])

question = "What is the capital of France?"

from langchain import HuggingFaceHub, LLMChain

hub_llm = HuggingFaceHub(repo_id = 'google/flan-t5-large', model_kwargs={'temperature': 0})

llm_chain = LLMChain(prompt=prompt, llm=hub_llm)

print(llm_chain.run(question))

c:\Users\veace\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\veace\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


paris


### Asking multiple questions
Iteratoing over questions

In [39]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)

for question, answer in zip(qa, res.generations):
    print(f'Question: {question["question"]}\nAnswer: {answer[0].text}\n')


Question: What is the capital city of France?
Answer: paris

Question: What is the largest mammal on Earth?
Answer: giraffe

Question: Which gas is most abundant in Earth's atmosphere?
Answer: nitrogen

Question: What color is a ripe banana?
Answer: yellow



Including multiple questions in the prompt

In [41]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""

long_prompt = PromptTemplate(template=multi_template, input_variables=['questions'])

llm_chain = LLMChain(prompt=long_prompt, llm=llm)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
	"What color is a ripe banana?\n"
)

llm_chain.run(qs_str)

'Paris\nBlue Whale\nNitrogen\nYellow'

In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature=0)

summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=['text'], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."
summarized_text = summarization_chain.predict(text=text)

summarized_text

'LangChain offers various modules for building language model applications, allowing users to combine them for more complex applications or use them individually for simpler ones, with the basic building block being calling an LLM on input, as demonstrated in the example of creating a company name based on its product.'

Text translation

In [45]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=['source_language', 'target_language', 'text'], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

source_language = "English"
target_language = "French"
text = "Your text here"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)

translated_text

'Votre texte ici'